In [ ]:
import pandas as pd
import ast

# Load datasets
powerbi_path = "powerbi_dataset.csv"
lyrics_path = "song_lyrics.csv"

df_powerbi = pd.read_csv(powerbi_path)
df_lyrics = pd.read_csv(lyrics_path)

# Print column names for debugging
print("Available columns in Power BI dataset:", df_powerbi.columns)
print("Available columns in Lyrics dataset:", df_lyrics.columns)

#  Rename columns in lyrics dataset to match Power BI dataset
df_lyrics.rename(columns={"artist": "Artist", "song_name": "Song"}, inplace=True)

#  Remove "Lyrics" suffix from artist names in `song_lyrics.csv`
df_lyrics["Artist"] = df_lyrics["Artist"].str.replace(" Lyrics", "", regex=False).str.strip()

#  Standardize text formatting to improve merging accuracy
df_powerbi["Artist"] = df_powerbi["Artist"].str.lower().str.strip()
df_powerbi["Song"] = df_powerbi["Song"].str.lower().str.strip()
df_lyrics["Artist"] = df_lyrics["Artist"].str.lower().str.strip()
df_lyrics["Song"] = df_lyrics["Song"].str.lower().str.strip()

#  Perform Exact Merge on Artist & Song
df_merged = df_powerbi.merge(df_lyrics[["Artist", "Song", "lyrics"]], on=["Artist", "Song"], how="left")

# 🔍 Check how many lyrics were successfully merged
matched_lyrics = df_merged["lyrics"].notna().sum()
print(f" Successfully merged {matched_lyrics} songs with lyrics.")

#  Fix Genre Column: Extract First Genre (or assign 'Other' if empty)
df_merged["Genre"] = df_merged["Genre"].replace(["", "[]"], pd.NA)

def extract_first_genre(Genre):
    """Extracts the full first genre or assigns 'Other' if empty."""
    if pd.isna(Genre):
        return "Other"
    try:
        genre_list = ast.literal_eval(Genre)  # Convert string to list
        if isinstance(genre_list, list) and len(genre_list) > 0:
            return genre_list[0]  # Take full first genre
    except (ValueError, SyntaxError):
        return Genre  # Keep original if conversion fails
    return "Other"

df_merged["Genre"] = df_merged["Genre"].apply(extract_first_genre)

#  Remove Duplicates
if "Artist" in df_merged.columns and "Song" in df_merged.columns:
    df_merged = df_merged.drop_duplicates(subset=["Artist", "Song"], keep="first")

#  Convert Popularity & Followers to Numeric
if "popularity" in df_merged.columns:
    df_merged["popularity"] = pd.to_numeric(df_merged["popularity"], errors="coerce").fillna(0)
if "followers" in df_merged.columns:
    df_merged["followers"] = pd.to_numeric(df_merged["followers"], errors="coerce").fillna(0)

#  Fill Missing Values
df_merged = df_merged.fillna({"followers": 0, "popularity": 0})  # Numeric fields filled with 0
df_merged = df_merged.fillna("Unknown")  # Text fields filled with "Unknown"

#  Standardize Sentiment Column (if exists)
if "sentiment" in df_merged.columns:
    df_merged["sentiment"] = df_merged["sentiment"].str.capitalize()

#  Remove Extra Whitespaces
df_merged = df_merged.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#  Save cleaned dataset with lyrics
cleaned_file_path = "powerbi_dataset_final_with_lyrics.csv"
df_merged.to_csv(cleaned_file_path, index=False)

print(" Data fully cleaned, genres fixed & lyrics added! Saved as 'powerbi_dataset_final_with_lyrics.csv'.")


Available columns in Power BI dataset: Index(['Artist', 'Song', 'Sentiment', 'Followers', 'Popularity', 'Genre'], dtype='object')
Available columns in Lyrics dataset: Index(['Unnamed: 0', 'link', 'artist', 'song_name', 'lyrics'], dtype='object')
✅ Successfully merged 32164 songs with lyrics.


C:\Users\91799\AppData\Local\Temp\ipykernel_33000\647929544.py:70: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_merged = df_merged.applymap(lambda x: x.strip() if isinstance(x, str) else x)


✅ Data fully cleaned, genres fixed & lyrics added! Saved as 'powerbi_dataset_final_with_lyrics.csv'.
